In [2]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import pandas as pd

In [5]:
from LSTM_model import NeuralNetwork
from run_model import compile_model,predict_model

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [7]:
df_data = pd.read_csv("bibim_learndata.csv")
df_data.columns

Index(['textLevel', 'textContent'], dtype='object')

In [14]:
from konlpy.tag import Okt
tokenizer = Okt()

AttributeError: Java package 'kr.lucypark.okt' is not valid

In [ ]:
class_num = len(df_data['textLevel'].unique())
data_len = 28
input_data = torch.rand(1, 28, 28) # 여기가 데이터 들어가는 곳

In [ ]:
model = compile_model(input_data,data_len,class_num)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)